# 🤖 Building AI Applications with LangChain: Complete Tutorial

Welcome to this comprehensive tutorial on building AI applications using LangChain! This notebook will guide you through creating increasingly complex AI applications, from basic chatbots to sophisticated agents.

## What You'll Learn

1. **Environment Setup** - Installing and configuring LangChain
2. **Basic LLM Integration** - Working with OpenAI and local models
3. **Prompt Engineering** - Advanced prompting techniques
4. **Memory Systems** - Maintaining conversation context
5. **Document Processing** - Loading and processing documents
6. **Vector Stores** - Embedding and similarity search
7. **RAG Systems** - Retrieval-Augmented Generation
8. **Custom Tools** - Building specialized functionality
9. **Agents** - Autonomous reasoning and action
10. **Production Features** - Streaming, monitoring, and deployment

Let's start building! 🚀

## 1. Environment Setup and Installation

First, let's set up our environment and install the necessary packages.

In [1]:
# Install required packages (run this if packages are not installed)
# !pip install langchain langchain-openai langchain-community python-dotenv faiss-cpu

# Import essential libraries
import os
import sys
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print("📦 LangChain Tutorial Environment")
print("✅ Libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")

# Check if API key is available
openai_key = os.getenv("OPENAI_API_KEY")
if openai_key:
    print("🔑 OpenAI API key found!")
else:
    print("⚠️  OpenAI API key not found. You can still use local models!")
    
print("\nReady to start building AI applications! 🚀")

📦 LangChain Tutorial Environment
✅ Libraries imported successfully!
🐍 Python version: 3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]
⚠️  OpenAI API key not found. You can still use local models!

Ready to start building AI applications! 🚀


## 2. Basic LLM Integration

Let's start with the basics - connecting to a Large Language Model and making simple API calls.

In [2]:
# Basic LLM setup - configured for your Ollama server
from langchain_ollama import OllamaLLM
# Commented out OpenAI import since we're using Ollama
# from langchain_openai import ChatOpenAI

def setup_llm(use_openai=False, model_name=None):
    """Setup LLM - configured to use your Ollama server by default."""
    # Your Ollama server configuration
    ollama_url = os.getenv("OLLAMA_BASE_URL", "http://192.168.71.5:11434")
    
    if use_openai and os.getenv("OPENAI_API_KEY"):
        # Using OpenAI (only if specifically requested and API key exists)
        from langchain_openai import ChatOpenAI
        model = model_name or "gpt-3.5-turbo"
        llm = ChatOpenAI(
            model=model,
            temperature=0.7,
            max_tokens=500,
        )
        print(f"🤖 Using OpenAI model: {model}")
        return llm
    else:
        # Using your local Ollama server (default)
        model = model_name or os.getenv("MODEL_NAME", "qwen3:latest")
        llm = OllamaLLM(
            model=model,
            base_url=ollama_url
        )
        print(f"🦙 Using Ollama model: {model} at {ollama_url}")
        return llm

# Initialize LLM - using your Ollama server
try:
    print("🔧 Setting up LLM connection...")
    llm = setup_llm(use_openai=False)  # Explicitly using Ollama
    
    # Test basic interaction with your server
    print("🧪 Testing connection with a simple question...")
    response = llm.invoke("Hello! Tell me a joke about programming.")
    print(f"\n🤖 AI Response:\n{response}")
    
except Exception as e:
    print(f"❌ Error setting up LLM: {e}")
    print("💡 Please check:")
    print("  1. Your Ollama server is running at http://192.168.71.5:11434")
    print("  2. The qwen3:latest model is available")
    print("  3. Your .env file has the correct OLLAMA_BASE_URL setting")

🔧 Setting up LLM connection...
🦙 Using Ollama model: qwen3:latest at http://192.168.71.5:11434
🧪 Testing connection with a simple question...

🤖 AI Response:
<think>
Okay, the user asked for a joke about programming. Let me think of a good one. I need to make sure it's appropriate and not too technical. Maybe something with common programming terms.

Hmm, there's the classic "Why do programmers prefer dark mode?" joke. Wait, that's been done before. Maybe something with debugging? Like the "debugger" pun. Or maybe a play on words with code terms.

Wait, here's an idea: "Why do programmers always mix up Halloween and Christmas? Because Oct 31 equals Dec 25!" Oh, that's a classic. But maybe the user has heard that one. Let me think of another.

What about the one with the programmer in a bar? "Why do programmers prefer dark mode? Because light mode is too bright and they can't see the errors!" No, that's similar to the first one. Maybe something with coffee and debugging?

Wait, here's a

## 3. Prompt Templates and Chain Basics

Prompt templates allow us to create reusable, dynamic prompts. Chains help us connect different components together.

In [3]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. Simple Prompt Template
simple_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms that a beginner could understand."
)

print("🎯 Simple Prompt Template:")
print(simple_prompt.format(topic="machine learning"))

# 2. Chat Prompt Template (for chat models)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI tutor specializing in {subject}."),
    ("human", "Please explain {topic} with examples.")
])

print("\n💬 Chat Prompt Template:")
formatted_prompt = chat_prompt.format_messages(
    subject="Python programming", 
    topic="list comprehensions"
)
for message in formatted_prompt:
    print(f"{message.type}: {message.content}")

# 3. Creating a Basic Chain
chain = chat_prompt | llm | StrOutputParser()

# Test the chain
try:
    result = chain.invoke({
        "subject": "data science",
        "topic": "the difference between supervised and unsupervised learning"
    })
    print(f"\n🔗 Chain Result:\n{result}")
except Exception as e:
    print(f"Error running chain: {e}")

🎯 Simple Prompt Template:
Explain machine learning in simple terms that a beginner could understand.

💬 Chat Prompt Template:
system: You are a helpful AI tutor specializing in Python programming.
human: Please explain list comprehensions with examples.

🔗 Chain Result:
<think>
Okay, the user is asking about the difference between supervised and unsupervised learning. Let me start by recalling the basics. Supervised learning uses labeled data, right? So the model learns from examples where the input and output are known. Examples include classification and regression tasks. Like predicting house prices based on features, which is regression. Or classifying emails as spam or not, which is classification.

Now, unsupervised learning doesn't have labels. The model tries to find patterns or structures in the data on its own. Clustering and dimensionality reduction are common here. For example, customer segmentation using clustering algorithms like K-means. Or maybe something like recommend

## 4. Memory Implementation

Memory allows our AI to remember previous conversations and maintain context across multiple interactions.

In [7]:
# Modern memory implementation using LangChain 1.0+
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

class MemoryBot:
    """A chatbot with conversation memory using your Ollama server - Modern LangChain 1.0+ approach."""
    
    def __init__(self, llm):
        self.llm = llm
        self.chat_history = []  # Simple list to store conversation history
        
        # Create prompt with memory placeholder
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant. Use the conversation history to provide context-aware responses."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}")
        ])
        
        # Create the chain
        self.chain = self.prompt | self.llm | StrOutputParser()
    
    def chat(self, message):
        """Send a message and get a response with memory."""
        # Invoke the chain with current history
        response = self.chain.invoke({
            "input": message,
            "chat_history": self.chat_history
        })
        
        # Add the new conversation to history
        self.chat_history.extend([
            HumanMessage(content=message),
            AIMessage(content=response)
        ])
        
        return response
    
    def get_memory(self):
        """Get current memory contents."""
        memory_str = ""
        for msg in self.chat_history:
            if isinstance(msg, HumanMessage):
                memory_str += f"Human: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                memory_str += f"AI: {msg.content}\n"
        return memory_str
    
    def clear_memory(self):
        """Clear conversation history."""
        self.chat_history = []

# Create memory-enabled chatbot using your Ollama server
print("🧠 Creating Memory-Enabled Chatbot with your Ollama server...")
memory_bot = MemoryBot(llm)

# Test conversation with memory
print("🧠 Testing Memory-Enabled Chatbot:")
print("-" * 40)

try:
    # First message
    response1 = memory_bot.chat("Hi, my name is Alice and I'm learning Python.")
    print(f"User: Hi, my name is Alice and I'm learning Python.")
    print(f"Bot: {response1}\n")

    # Second message - bot should remember the name
    response2 = memory_bot.chat("What programming concepts should I focus on first?")
    print(f"User: What programming concepts should I focus on first?")
    print(f"Bot: {response2}\n")

    # Third message - testing memory retention
    response3 = memory_bot.chat("What was my name again?")
    print(f"User: What was my name again?")
    print(f"Bot: {response3}\n")

    print("📝 Current Memory Buffer:")
    print(memory_bot.get_memory())
    
except Exception as e:
    print(f"❌ Error in memory chat: {e}")
    print("💡 Make sure your Ollama server is running and accessible")

🧠 Creating Memory-Enabled Chatbot with your Ollama server...
🧠 Testing Memory-Enabled Chatbot:
----------------------------------------
User: Hi, my name is Alice and I'm learning Python.
Bot: <think>
Okay, the user introduced herself as Alice and mentioned she's learning Python. I need to respond in a friendly and helpful manner. Let me start by welcoming her and expressing enthusiasm that she's learning Python. Maybe ask how she's finding the learning process so far. It's important to offer assistance with any specific questions or topics she might be working on. I should keep the tone positive and encouraging. Let me make sure the response is concise but welcoming, and invite her to ask for help if needed. Avoid any technical jargon since she's just starting out. Alright, that should cover it.
</think>

Hello, Alice! Welcome to the world of Python programming. I'm glad to hear you're learning Python—it's a fantastic language for beginners! How are you finding the learning process so